Importing all the necessary items

In [ ]:
# Bring in github repositories
!pip install --upgrade git+https://github.com/edgi-govdata-archiving/ECHO_modules &>/dev/null;
!pip install geopandas &>/dev/null;
!pip install folium &>/dev/null;

if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/edgi-govdata-archiving/EEW_County_ReportCards.git
    %mv /content/EEW_County_ReportCards/Region.py /content
    %mv /content/EEW_County_ReportCards/AllPrograms_util.py /content
    %mv /content/EEW_County_ReportCards/region.db /content
else:
    print('Not running on CoLab')

# Import libraries
import pdb
import os
import pandas as pd
import sqlite3
from AllPrograms_util import get_region_rowid
from Region import Region

Running on CoLab
Cloning into 'EEW_County_ReportCards'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 27 (delta 7), reused 18 (delta 5), pack-reused 0
Receiving objects: 100% (27/27), 20.77 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (7/7), done.



(Number I) : from county_per_1000, how many counties with below average median inspections per 1000 facilities (CWA, RCRA, CAA) (and where?) (below average in reference to the nationwide rate and below average in reference to that state's statewide rate, available)

In [ ]:
def natl_avg_calc(facility):
  '''
  RETURNS the national average of inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA)
  '''
  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Insp.per.1000"
  sql = f"select avg([{column}]) from county_per_1000"

  cur.execute(sql)
  avg = cur.fetchone()[0]

  cur.close()
  con.close()
  return avg

CAA_national_average = natl_avg_calc("CAA")
CWA_national_average = natl_avg_calc("CWA")
RCRA_national_average = natl_avg_calc("RCRA")

def state_avg_calc(facility):
  '''
  RETURNS the state average of inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA)
  '''
  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Insp.per.1000"
  sql = f"select [CD.State] as state, avg([{column}]) as average from state_per_1000 group by state;"
  cur.execute(sql)
  rows = cur.fetchall()

  cur.close()
  con.close()
  return {row[0]: row[1] for row in rows}

CAA_state_averages_map = state_avg_calc("CAA")
CWA_state_averages_map = state_avg_calc("CWA")
RCRA_state_averages_map = state_avg_calc("RCRA")

def less_than_natl(facility, avg):
  '''
  RETURNS the number of facilities that are less than national average of
  inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA) and avg be a float of the national average.
  '''
  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Insp.per.1000"
  sql = f"select count(*) from county_per_1000 where [{column}] < {avg}"
  cur.execute(sql)
  count = cur.fetchone()[0]

  sql = f"select [CD.State] from county_per_1000 where [{column}] < {avg}"
  cur.execute(sql)
  list_of_counties = [county for county in cur]

  cur.close()
  con.close()
  return count, list_of_counties

CAA_less_than_natl = less_than_natl("CAA", CAA_national_average)
print("The number of counties with below average median inspections per 1000 CAA facilities with respect to the national average is " + str(CAA_less_than_natl[0]))
print(f"These counties are {CAA_less_than_natl[1]}",end='\n\n')
CWA_less_than_natl = less_than_natl("CWA", CWA_national_average)
print("The number of counties with below average median inspections per 1000 CWA facilities with respect to the national average is " + str(CWA_less_than_natl[0]))
print(f"These counties are {CWA_less_than_natl[1]}",end='\n\n')
RCRA_less_than_natl = less_than_natl("RCRA", RCRA_national_average)
print("The number of counties with below average median inspections per 1000 RCRA facilities with respect to the national average is " + str(RCRA_less_than_natl[0]))
print(f"These counties are {RCRA_less_than_natl[1]}",end='\n\n')

def less_than_state(facility, avg):

  '''
  RETURNS the number of facilities that are less than state average of
  inspections per 1000 facilities for whatever state the county is in.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA) and avg be a map of states to their average inspections per 1000 facilities.
  '''

  con = sqlite3.connect("region.db")
  cur = con.cursor()
  count = 0
  for key in avg:
    val = avg[key]
    column = facility + ".Insp.per.1000"
    sql = f"select count(*) from county_per_1000 where [{column}] < {val} and substr([CD.state], 1, 2) = '{key}'"
    cur.execute(sql)
    state_count = cur.fetchone()[0]
    count += state_count


    sql = f"select [CD.State] from county_per_1000 where [{column}] < {val} and substr([CD.state], 1, 2) = '{key}'"
    cur.execute(sql)
    list_of_counties = [county for county in cur]
    print(list_of_counties)

  cur.close()
  con.close()
  return count

CAA_less_than_state = less_than_state("CAA", CAA_state_averages_map)
print("The number of counties with below average median inspections per 1000 CAA facilities with respect to the state average is " + str(CAA_less_than_state),end="\n\n")
CWA_less_than_state = less_than_state("CWA", CWA_state_averages_map)
print("The number of counties with below average median inspections per 1000 CWA facilities with respect to the state average is " + str(CWA_less_than_state),end="\n\n")
RCRA_less_than_state = less_than_state("RCRA", RCRA_state_averages_map)
print("The number of counties with below average median inspections per 1000 RCRA facilities with respect to the state average is " + str(RCRA_less_than_state),end="\n\n")

The number of counties with below average median inspections per 1000 CAA facilities with respect to the national average is 2941
The number of counties with below average median inspections per 1000 CWA facilities with respect to the national average is 3243
The number of counties with below average median inspections per 1000 RCRA facilities with respect to the national average is 3053
The number of counties with below average median inspections per 1000 CAA facilities with respect to the state average is 1758
The number of counties with below average median inspections per 1000 CWA facilities with respect to the state average is 1151
The number of counties with below average median inspections per 1000 RCRA facilities with respect to the state average is 1022


(Number II): from county_per_1000, how many counties with below average median enforcement actions per 1000 facilities (CWA, RCRA, CAA) (and where?) (below average in reference to the nationwide rate and below average in reference to that state's statewide rate, available from state_per_1000)

In [ ]:
def natl_avg_calc_enf(facility):
  '''
  RETURNS the national average of inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA)
  '''
  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Enf.per.1000"
  sql = f"select avg([{column}]) from county_per_1000"

  cur.execute(sql)
  avg = cur.fetchone()[0]
  

  cur.close()
  con.close()
  return avg

CAA_national_average_enf = natl_avg_calc_enf("CAA")
CWA_national_average_enf = natl_avg_calc_enf("CWA")
RCRA_national_average_enf = natl_avg_calc_enf("RCRA")

def less_than_natl_enf(facility, avg):

  '''
  RETURNS the number of facilities that are less than national average of
  inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA) and avg be a float of the national average.
  '''

  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Enf.per.1000"
  sql = f"select count(*) from county_per_1000 where [{column}] < {avg}"
  cur.execute(sql)
  count = cur.fetchone()[0]

  sql = f"select [CD.State] from county_per_1000 where [{column}] < {avg}"
  cur.execute(sql)
  list_of_counties = [county for county in cur]

  cur.close()
  con.close()
  return count, list_of_counties

CAA_less_than_natl_enf = less_than_natl_enf("CAA", CAA_national_average_enf)
print("The number of counties with below average median enforcements per 1000 CAA facilities with respect to the national average is " + str(CAA_less_than_natl_enf))
CWA_less_than_natl_enf = less_than_natl_enf("CWA", CWA_national_average_enf)
print("The number of counties with below average median enforcements per 1000 CWA facilities with respect to the national average is " + str(CWA_less_than_natl_enf))
RCRA_less_than_natl_enf = less_than_natl_enf("RCRA", RCRA_national_average_enf)
print("The number of counties with below average median enforcements per 1000 RCRA facilities with respect to the national average is " + str(RCRA_less_than_natl_enf))

def state_avg_calc_enf(facility):
  '''
  RETURNS the state average of inspections per 1000 facilities.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA)
  '''
  con = sqlite3.connect("region.db")
  cur = con.cursor()

  column = facility + ".Enf.per.1000"
  sql = f"select [CD.State] as state, avg([{column}]) as average from state_per_1000 group by state;"
  cur.execute(sql)
  rows = cur.fetchall()

  cur.close()
  con.close()
  return {row[0]: row[1] for row in rows}

CAA_state_averages_map_enf = state_avg_calc_enf("CAA")
CWA_state_averages_map_enf = state_avg_calc_enf("CWA")
RCRA_state_averages_map_enf = state_avg_calc_enf("RCRA")

def less_than_state_enf(facility, avg):

  '''
  RETURNS the number of facilities that are less than state average of
  enforcement actions per 1000 facilities for whatever state the county is in.

  REQUIRES facility be the string representation type of facility to sort
  (CWA, RCRA, CAA) and avg be a map of states to their average enforcement actions per 1000 facilities.
  '''


  con = sqlite3.connect("region.db")
  cur = con.cursor()

  count = 0
  for key in avg:
    val = avg[key]
    column = facility + ".Enf.per.1000"
    sql = f"select count(*) from county_per_1000 where [{column}] < {val} and substr([CD.state], 1, 2) = '{key}'"
    cur.execute(sql)
    state_count = cur.fetchone()[0]
    count += state_count

    sql = f"select [CD.State] from county_per_1000 where [{column}] < {val} and substr([CD.state], 1, 2) = '{key}'"
    cur.execute(sql)
    list_of_counties = [county for county in cur]
    print(list_of_counties)

  cur.close()
  con.close()
  return count

CAA_less_than_state_enf = less_than_state_enf("CAA", CAA_state_averages_map_enf)
print("The number of counties with below average median enforcements per 1000 CAA facilities with respect to the state average is " + str(CAA_less_than_state_enf),end="\n\n")
CWA_less_than_state_enf = less_than_state_enf("CWA", CWA_state_averages_map_enf)
print("The number of counties with below average median enforcements per 1000 CWA facilities with respect to the state average is " + str(CWA_less_than_state_enf),end="\n\n")
RCRA_less_than_state_enf = less_than_state_enf("RCRA", RCRA_state_averages_map_enf)
print("The number of counties with below average median enforcements per 1000 RCRA facilities with respect to the state average is " + str(RCRA_less_than_state_enf),end="\n\n")

The number of counties with below average median enforcements per 1000 CAA facilities with respect to the national average is 3157
The number of counties with below average median enforcements per 1000 CWA facilities with respect to the national average is 3088
The number of counties with below average median enforcements per 1000 RCRA facilities with respect to the national average is 3002
The number of counties with below average median enforcements per 1000 CAA facilities with respect to the state average is 1910
The number of counties with below average median enforcements per 1000 CWA facilities with respect to the state average is 1318
The number of counties with below average median enforcements per 1000 RCRA facilities with respect to the state average is 1074


(Number III): from violations_by_facilities, how many counties with a large share of facilities(greater than 75% of the other counties) with 12 or 13 noncomp quarters (quarters in non-compliance)?

In [ ]:
conn = sqlite3.connect("region.db")
cur = conn.cursor()

sql = 'select sum(noncomp_qtrs = 12 or noncomp_qtrs = 13)'
sql += ' from (select noncomp_qtrs from violations_by_facilities'
sql += ' order by num_facilities desc'
sql += ' limit (select count(*)/4 from violations_by_facilities)) as subquery'

cur.execute(sql)
result = cur.fetchone()[0]
cur.close()
conn.close()
print("The number of counties with a large share of facilites with 12 or 13 noncomp quarteres is " + str(result))

The number of counties with a large share of facilites with 12 or 13 noncomp quarteres is 493
